In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.metrics import silhouette_score
import pandas as pd
import warnings
import re
from sklearn.metrics import pairwise_distances_argmin_min
import seaborn as sns

ModuleNotFoundError: No module named 'geopandas'

##데이터 전처리 및 병합

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [ ]:
!pip install openpyxl

(1) 고령자 인구 & 노인 자살률

In [ ]:
suicide_rate_path = "/content/drive/MyDrive/코랩/데이터마이닝/자살률(구별).xlsx"

# 전국 평균 자살률 정보 (단위: 명 / 10만명)
national_elderly_suicide_rate = 40.6
national_total_suicide_rate = 27.3

# 자치구별 자살률 데이터 불러오기 및 0행 제거
df_raw = pd.read_excel(suicide_rate_path)
df = df_raw.iloc[1:].copy()  # 0행(소계) 제거

# 필요한 컬럼 추출 및 이름 변경
df = df[['자치구별', '자살률 (10만명당 명)']]
df.rename(columns={
    '자치구별': '자치구',
    '자살률 (10만명당 명)': '전체 자살률'
}, inplace=True)

# 3️자치구별 추정 노인 자살률 열 추가
df['추정 노인 자살률'] = df['전체 자살률'] * (national_elderly_suicide_rate / national_total_suicide_rate)

df = df[['자치구', '추정 노인 자살률']]

df

,자치구,추정 노인 자살률
1,종로구,36.435897
2,중구,43.723077
3,용산구,32.271795
4,성동구,34.353846
5,광진구,40.451282
6,동대문구,37.179487
7,중랑구,40.600000
8,성북구,33.164103
9,강북구,37.030769
10,도봉구,41.194872


(2) 독거 노인 인구

In [ ]:
lonely_df = pd.read_excel("/content/drive/MyDrive/코랩/데이터마이닝/팀플/data/독거 노인 인구.xlsx")

lonely_cleaned = lonely_df.iloc[1:].copy()

# 필요한 열 선택 및 열 이름 변경
lonely_cleaned = lonely_cleaned[['자치구', '소계', '국민기초생활보장 수급권자', '저소득노인']]
lonely_cleaned.columns = ['자치구', '독거노인 합계', '기초수급자 합계', '저소득노인 합계']

# 수치형으로 변환
lonely_cleaned[['독거노인 합계', '기초수급자 합계', '저소득노인 합계']] = lonely_cleaned[
    ['독거노인 합계', '기초수급자 합계', '저소득노인 합계']
].apply(pd.to_numeric, errors='coerce')

# df에 중복 열이 있을 경우 삭제
for col in ['독거노인 합계', '기초수급자 합계', '저소득노인 합계']:
    if col in df.columns:
        df = df.drop(columns=[col])

# 자치구 기준으로 병합
df = pd.merge(df, lonely_cleaned, on='자치구', how='left')

df

,자치구,추정 노인 자살률,독거노인 합계,기초수급자 합계,저소득노인 합계
0,종로구,36.435897,11178,2231,403
1,중구,43.723077,7987,2553,324
2,용산구,32.271795,10498,3029,96
3,성동구,34.353846,12141,3536,239
4,광진구,40.451282,13657,3875,981
5,동대문구,37.179487,18625,5993,446
6,중랑구,40.600000,22438,8428,391
7,성북구,33.164103,19763,7495,994
8,강북구,37.030769,20398,7635,767
9,도봉구,41.194872,17136,4964,1133


(3) 노인 의료복지 시설

In [ ]:
medical_df = pd.read_excel("/content/drive/MyDrive/코랩/데이터마이닝/팀플/data/서울시 노인의료복지시설현황.xlsx")

facilities = medical_df.iloc[3:].copy()

# 자치구 열 이름 지정 및 정리
facilities = facilities.rename(columns={'Unnamed: 1': '자치구'})
facilities['자치구'] = facilities['자치구'].astype(str).str.strip()

facilities = facilities[facilities['자치구'].str.contains("구", na=False)]

# 자치구별 시설 수 계산
facility_counts = facilities['자치구'].value_counts().reset_index()
facility_counts.columns = ['자치구', '노인의료복지시설 수']

df['자치구'] = df['자치구'].astype(str).str.strip()

# 병합 전 동일 열이 있으면 제거
if '노인의료복지시설 수' in df.columns:
    df = df.drop(columns=['노인의료복지시설 수'])

# 병합
df = pd.merge(df, facility_counts, on='자치구', how='left')

# 9. NaN → 0 처리
df['노인의료복지시설 수'] = df['노인의료복지시설 수'].fillna(0).astype(int)

df

,자치구,추정 노인 자살률,독거노인 합계,기초수급자 합계,저소득노인 합계,노인의료복지시설 수
0,종로구,36.435897,11178,2231,403,8
1,중구,43.723077,7987,2553,324,2
2,용산구,32.271795,10498,3029,96,2
3,성동구,34.353846,12141,3536,239,4
4,광진구,40.451282,13657,3875,981,8
5,동대문구,37.179487,18625,5993,446,7
6,중랑구,40.600000,22438,8428,391,11
7,성북구,33.164103,19763,7495,994,19
8,강북구,37.030769,20398,7635,767,10
9,도봉구,41.194872,17136,4964,1133,21


(4) 재가노인 복지시설

In [ ]:
homecare_df = pd.read_excel("/content/drive/MyDrive/코랩/데이터마이닝/팀플/data/서울시 재가노인 복지시설 현황.xlsx")

# 자치구와 시설 수(합계) 열만 추출
homecare_facilities = homecare_df[['자치구', '합계']].copy()

homecare_facilities['자치구'] = homecare_facilities['자치구'].astype(str).str.strip()

homecare_facilities['합계'] = pd.to_numeric(homecare_facilities['합계'], errors='coerce')

# 열 이름 변경
homecare_facilities = homecare_facilities.rename(columns={'합계': '재가노인 복지시설 수(개소)'})

df['자치구'] = df['자치구'].astype(str).str.strip()

# 병합 전 동일 열이 있으면 제거
if '재가노인 복지시설 수(개소)' in df.columns:
    df = df.drop(columns=['재가노인 복지시설 수(개소)'])

df = pd.merge(df, homecare_facilities, on='자치구', how='left')

# NaN 값은 0으로 처리
df['재가노인 복지시설 수(개소)'] = df['재가노인 복지시설 수(개소)'].fillna(0).astype(int)

# 불필요한 열 제거
if '재가노인 복지시설 수' in df.columns:
    df = df.drop(columns=['재가노인 복지시설 수'])

# 열 이름 변경
df = df.rename(columns={'재가노인 복지시설 수(개소)': '재가노인복지시설 수'})
df

,자치구,추정 노인 자살률,독거노인 합계,기초수급자 합계,저소득노인 합계,노인의료복지시설 수,재가노인복지시설 수
0,종로구,36.435897,11178,2231,403,8,27
1,중구,43.723077,7987,2553,324,2,34
2,용산구,32.271795,10498,3029,96,2,21
3,성동구,34.353846,12141,3536,239,4,43
4,광진구,40.451282,13657,3875,981,8,55
5,동대문구,37.179487,18625,5993,446,7,87
6,중랑구,40.600000,22438,8428,391,11,169
7,성북구,33.164103,19763,7495,994,19,92
8,강북구,37.030769,20398,7635,767,10,109
9,도봉구,41.194872,17136,4964,1133,21,122


(5) 여가복지시설

In [ ]:
leisure_df = pd.read_excel("/content/drive/MyDrive/코랩/데이터마이닝/팀플/data/서울시 노인여가 복지시설 현황 .xlsx")

# 필요한 열만 추출하고 이름 정리
leisure_df = leisure_df[['자치구', '노인복지관 수', '복지관종사자 수', '경로당 수', '노인교실 수']]
leisure_df.rename(columns={
    '노인복지관 수': '복지관 수',
    '복지관종사자 수': '복지관종사자 수',
    '경로당 수': '경로당 수',
    '노인교실 수': '노인교실 수'
}, inplace=True)

# df에 네 가지 열을 자치구 기준으로 붙이기
for col in ['복지관 수', '복지관종사자 수', '경로당 수', '노인교실 수']:
    df[col] = df['자치구'].map(leisure_df.set_index('자치구')[col])

df

,자치구,추정 노인 자살률,독거노인 합계,기초수급자 합계,저소득노인 합계,노인의료복지시설 수,재가노인복지시설 수,복지관 수,복지관종사자 수,경로당 수,노인교실 수
0,종로구,36.435897,11178,2231,403,8,27,3,169,62,9
1,중구,43.723077,7987,2553,324,2,34,2,71,50,3
2,용산구,32.271795,10498,3029,96,2,21,2,84,89,20
3,성동구,34.353846,12141,3536,239,4,43,5,104,164,12
4,광진구,40.451282,13657,3875,981,8,55,1,63,97,8
5,동대문구,37.179487,18625,5993,446,7,87,1,114,136,16
6,중랑구,40.600000,22438,8428,391,11,169,5,128,131,11
7,성북구,33.164103,19763,7495,994,19,92,6,112,180,7
8,강북구,37.030769,20398,7635,767,10,109,1,36,100,18
9,도봉구,41.194872,17136,4964,1133,21,122,5,102,136,4


(6) 요양 보호사 수

In [ ]:
caregiver_df = pd.read_csv("/content/drive/MyDrive/코랩/데이터마이닝/팀플/data/서울시 요양보호사 남녀별 자격 현황정보.csv", encoding='cp949')

gu_names = caregiver_df['자치구명'].unique()
print("자치구명 목록:")
for gu in gu_names:
    print(gu)

자치구명 목록:
타시도
강동구
강북구
서대문구
마포구
강남구
광진구
은평구
동작구
중구
종로구
도봉구
양천구
금천구
구로구
영등포구
성동구
관악구
강서구
동대문구
송파구
성북구
용산구
노원구
서초구


중랑구 데이터 결측 확인

In [ ]:
caregiver_by_gu = caregiver_df.groupby('자치구명')['인원(명)'].sum().reset_index()
caregiver_by_gu.rename(columns={'자치구명': '자치구', '인원(명)': '요양보호사 수'}, inplace=True)

# 기존 df에 자치구 기준으로 병합
df = pd.merge(df, caregiver_by_gu, on='자치구', how='left')

df

,자치구,추정 노인 자살률,독거노인 합계,기초수급자 합계,저소득노인 합계,노인의료복지시설 수,재가노인복지시설 수,복지관 수,복지관종사자 수,경로당 수,노인교실 수,요양보호사 수
0,종로구,36.435897,11178,2231,403,8,27,3,169,62,9,4986.0
1,중구,43.723077,7987,2553,324,2,34,2,71,50,3,4081.0
2,용산구,32.271795,10498,3029,96,2,21,2,84,89,20,6151.0
3,성동구,34.353846,12141,3536,239,4,43,5,104,164,12,10017.0
4,광진구,40.451282,13657,3875,981,8,55,1,63,97,8,15103.0
5,동대문구,37.179487,18625,5993,446,7,87,1,114,136,16,14132.0
6,중랑구,40.600000,22438,8428,391,11,169,5,128,131,11,NaN
7,성북구,33.164103,19763,7495,994,19,92,6,112,180,7,18916.0
8,강북구,37.030769,20398,7635,767,10,109,1,36,100,18,16496.0
9,도봉구,41.194872,17136,4964,1133,21,122,5,102,136,4,18300.0


In [ ]:
# 요양보호사 수의 평균으로 중랑구 결측치 채우기
mean_caregiver = df['요양보호사 수'].mean()
df.loc[df['자치구'] == '중랑구', '요양보호사 수'] = mean_caregiver

(7) 정류장 & 평균 노선 수

In [ ]:
bus_stop_df = pd.read_excel("/content/drive/MyDrive/코랩/데이터마이닝/팀플/data/서울시 정류소현황(2019~2023년).xlsx")

# 자치구별로 정류장 수 (고유 ARS-ID 개수)와 평균 노선 수 계산
bus_stats = bus_stop_df.groupby('행정구명').agg(
    정류장수=('ARS-ID', 'nunique'),
    평균노선수=('노선수', 'mean')
).reset_index()

# 열 이름 정리
bus_stats.rename(columns={'행정구명': '자치구'}, inplace=True)


# 기존 df에 병합
df = pd.merge(df, bus_stats, on='자치구', how='left')

df

,자치구,추정 노인 자살률,독거노인 합계,기초수급자 합계,저소득노인 합계,노인의료복지시설 수,재가노인복지시설 수,복지관 수,복지관종사자 수,경로당 수,노인교실 수,요양보호사 수,정류장수,평균노선수
0,종로구,36.435897,11178,2231,403,8,27,3,169,62,9,4986.00,386,3.481865
1,중구,43.723077,7987,2553,324,2,34,2,71,50,3,4081.00,229,5.454148
2,용산구,32.271795,10498,3029,96,2,21,2,84,89,20,6151.00,351,3.048433
3,성동구,34.353846,12141,3536,239,4,43,5,104,164,12,10017.00,447,2.118568
4,광진구,40.451282,13657,3875,981,8,55,1,63,97,8,15103.00,294,3.414966
5,동대문구,37.179487,18625,5993,446,7,87,1,114,136,16,14132.00,322,4.593168
6,중랑구,40.600000,22438,8428,391,11,169,5,128,131,11,16079.75,380,4.368421
7,성북구,33.164103,19763,7495,994,19,92,6,112,180,7,18916.00,604,2.657285
8,강북구,37.030769,20398,7635,767,10,109,1,36,100,18,16496.00,451,2.689579
9,도봉구,41.194872,17136,4964,1133,21,122,5,102,136,4,18300.00,391,2.826087


(8) 노인 보행 교통사고

In [ ]:
traffic_data = pd.read_excel('/content/drive/MyDrive/코랩/데이터마이닝/팀플/data/서울시 노인 교통사고 현황 통계.xlsx', sheet_name='데이터')

# 필요한 열만 선택
traffic_data_selected = traffic_data[['자치구', '노인 보행 교통사고']]

df = df.merge(traffic_data_selected, on='자치구', how='left')

df

,자치구,추정 노인 자살률,독거노인 합계,기초수급자 합계,저소득노인 합계,노인의료복지시설 수,재가노인복지시설 수,복지관 수,복지관종사자 수,경로당 수,노인교실 수,요양보호사 수,정류장수,평균노선수,노인 보행 교통사고
0,종로구,36.435897,11178,2231,403,8,27,3,169,62,9,4986.00,386,3.481865,67
1,중구,43.723077,7987,2553,324,2,34,2,71,50,3,4081.00,229,5.454148,60
2,용산구,32.271795,10498,3029,96,2,21,2,84,89,20,6151.00,351,3.048433,31
3,성동구,34.353846,12141,3536,239,4,43,5,104,164,12,10017.00,447,2.118568,52
4,광진구,40.451282,13657,3875,981,8,55,1,63,97,8,15103.00,294,3.414966,23
5,동대문구,37.179487,18625,5993,446,7,87,1,114,136,16,14132.00,322,4.593168,155
6,중랑구,40.600000,22438,8428,391,11,169,5,128,131,11,16079.75,380,4.368421,122
7,성북구,33.164103,19763,7495,994,19,92,6,112,180,7,18916.00,604,2.657285,97
8,강북구,37.030769,20398,7635,767,10,109,1,36,100,18,16496.00,451,2.689579,80
9,도봉구,41.194872,17136,4964,1133,21,122,5,102,136,4,18300.00,391,2.826087,61


In [ ]:
# df를 엑셀 파일로 저장
output_path = "/content/drive/MyDrive/코랩/데이터마이닝/팀플/전처리데이터.xlsx"
df.to_excel(output_path, index=False)

print(f"엑셀 파일이 저장되었습니다: {output_path}")

엑셀 파일이 저장되었습니다: /content/drive/MyDrive/코랩/데이터마이닝/팀플/전처리데이터.xlsx


In [ ]:
# 기존 전처리 데이터
df = pd.read_excel("/content/drive/MyDrive/코랩/데이터마이닝/팀플/전처리데이터.xlsx")

# 고령자 등록 인구 데이터
elderly_df = pd.read_excel("/content/drive/MyDrive/코랩/데이터마이닝/팀플/data/고령자 등록인구.xlsx")

# 자치구 기준으로 병합
merged_df = df.merge(elderly_df, on='자치구', how='left')

# 결과 파일로 저장
merged_df.to_excel("/content/drive/MyDrive/코랩/데이터마이닝/팀플/전처리데이터_노인인구포함.xlsx", index=False)

print("✅ 병합 완료: '전처리데이터_고령자포함.xlsx' 파일이 생성되었습니다.")

✅ 병합 완료: '전처리데이터_고령자포함.xlsx' 파일이 생성되었습니다.
